In [37]:
import numpy as np
from sklearn.datasets import make_sparse_spd_matrix
from scipy import linalg as LA
import pandas as pd
import time, os
import matplotlib.pyplot as plt

from pyinstrument import Profiler

from infoband.band_info import InfoCorrBand
from wlpy.covariance import Covariance
from utils.adpt_correlation_threshold import AdptCorrThreshold
from wlpy.gist import heatmap

import warnings
warnings.filterwarnings("ignore")

from my_api import *

# data processing

Use the error rate as our metric.
$$ \frac{\| A - \hat A \|}{\| A \|} $$

### Results of other methods

In [38]:
df = pd.read_csv('other_methods.csv')
df.drop(columns = df.columns[0], inplace = True)
df.head(3)

,N,T,rho,ord,type,Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink
0,100,100,0.8,fro,R,0.437159,0.412584,0.437159,0.405599,NaN
1,100,100,0.8,fro,S,0.468781,0.423442,0.468781,0.419056,NaN
2,100,100,0.8,2,R,0.419901,0.411377,0.348096,0.455421,NaN


### Results of our proposed methods

In [39]:
data_path = 'data_2023-01-17/'

In [40]:
def compute_error(row, eta):
    N, T, rho, ord, type = row['N'], row['T'], row['rho'], row['ord'], row['type']
    if type == 'R':
        cov_cor = 'cor'
    else:
        cov_cor = 'cov'
    file_name = get_title_1(ord = ord, cov_cor = cov_cor, eta = eta, N = N, T = T, rho = rho, draw_type = 'random')
    file_path = data_path + file_name + '.txt'
    try:
        data = np.loadtxt(file_path)
        return data.mean()
    except FileNotFoundError as e:
        return None

In [41]:
eta_range = [0.5, 0.8, 0.95, 1]
for eta in eta_range:
    df['Info Band eta={}'.format(eta)] = df.apply(func = compute_error, axis = 1, args = (eta, ))
df.head(3)

,N,T,rho,ord,type,Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink,Info Band eta=0.5,Info Band eta=0.8,Info Band eta=0.95,Info Band eta=1
0,100,100,0.8,fro,R,0.437159,0.412584,0.437159,0.405599,NaN,0.423696,0.330569,0.225972,0.226556
1,100,100,0.8,fro,S,0.468781,0.423442,0.468781,0.419056,NaN,0.449804,0.365138,0.276803,0.277256
2,100,100,0.8,2,R,0.419901,0.411377,0.348096,0.455421,NaN,0.363267,0.298962,0.237871,0.238219


# find conditions with best performance

Given (N, T, rho, ord, type), find the method with lowest error rate in the list $other\_cols$, which is defined as:

In [42]:
other_cols = ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']

In [43]:
# type(row) = pandas.Series, with indexes ['Sample', 'Soft Threshold', ...]
get_min_key = lambda row: min(row.index, key = lambda x: row[x]) 
df['min_other_key'] = df[other_cols].apply(func = get_min_key, axis = 1)
get_min_val = lambda row: min(row)
df['min_other_val'] = df[other_cols].apply(func = get_min_val, axis = 1)
df['min_other_key'].value_counts()

Soft Threshold      39
Linear Shrink       12
Hard Threshold      12
Nonlinear Shrink     7
Sample               2
Name: min_other_key, dtype: int64

### Compute ratios

For each row, given $\eta$, use the ratio 
$$ \frac{\text{lowest error rate of other methods (i.e. }min\_other\_val)}{\text{error rate of the proposed estimator}} $$

as a rule to find the parameters (N, T, rho) with best results.

In [44]:
for eta in eta_range:
    df['ratio_eta={}'.format(eta)] = df['min_other_val'] / df['Info Band eta={}'.format(eta)]
df.head(3)

,N,T,rho,ord,type,Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink,Info Band eta=0.5,Info Band eta=0.8,Info Band eta=0.95,Info Band eta=1,min_other_key,min_other_val,ratio_eta=0.5,ratio_eta=0.8,ratio_eta=0.95,ratio_eta=1
0,100,100,0.8,fro,R,0.437159,0.412584,0.437159,0.405599,NaN,0.423696,0.330569,0.225972,0.226556,Linear Shrink,0.405599,0.957286,1.226973,1.794911,1.790279
1,100,100,0.8,fro,S,0.468781,0.423442,0.468781,0.419056,NaN,0.449804,0.365138,0.276803,0.277256,Linear Shrink,0.419056,0.931642,1.147666,1.513913,1.511442
2,100,100,0.8,2,R,0.419901,0.411377,0.348096,0.455421,NaN,0.363267,0.298962,0.237871,0.238219,Hard Threshold,0.348096,0.958236,1.164347,1.463378,1.461242


### Sort the computed ratio

In [47]:
df.shape

(72, 20)

In [53]:
param_cols = ['N', 'T', 'rho']
ratio_cols = ['ratio_eta={}'.format(eta) for eta in eta_range]
print(ratio_cols)

['ratio_eta=0.5', 'ratio_eta=0.8', 'ratio_eta=0.95', 'ratio_eta=1']


In [59]:
for eta in eta_range:
    sort_col = 'ratio_eta={}'.format(eta)
    for ord in ['fro', '2']:
        for type in ['R', 'S']:
            print(ord, type, eta, sep = ' ')
            tmp = df[(df['ord'] == ord) & (df['type'] == type)] 
            tmp = tmp.sort_values(by = sort_col, ascending = False)[param_cols + ratio_cols]
            print(tmp.head(2))

fro R 0.5
      N    T   rho  ratio_eta=0.5  ratio_eta=0.8  ratio_eta=0.95  ratio_eta=1
48  500  100  0.80       1.087016       1.412228        1.847649     1.800520
20  100  500  0.95       0.979090       0.979090        0.795588     0.762526
fro S 0.5
      N    T  rho  ratio_eta=0.5  ratio_eta=0.8  ratio_eta=0.95  ratio_eta=1
49  500  100  0.8       1.058358       1.312218        1.598181     1.570540
41  300  500  0.8       0.942082       1.335859        2.216500     2.361762
2 R 0.5
      N    T   rho  ratio_eta=0.5  ratio_eta=0.8  ratio_eta=0.95  ratio_eta=1
50  500  100  0.80       1.467744       1.983244        2.281038     2.208524
54  500  100  0.95       1.397420       1.984727        2.492834     2.607874
2 S 0.5
      N    T  rho  ratio_eta=0.5  ratio_eta=0.8  ratio_eta=0.95  ratio_eta=1
35  300  300  0.8       1.415961       1.727682        1.525635     1.477475
27  300  100  0.8       1.368773       1.595019        1.332332     1.332332
fro R 0.8
      N    T  rho  ratio

# Query

In [64]:
tmp = df[(df['ord'] == 'fro') & (df['type'] == 'S') & (df['N'] == 500)]
tmp

,N,T,rho,ord,type,Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink,Info Band eta=0.5,Info Band eta=0.8,Info Band eta=0.95,Info Band eta=1,min_other_key,min_other_val,ratio_eta=0.5,ratio_eta=0.8,ratio_eta=0.95,ratio_eta=1
49,500,100,0.80,fro,S,1.047401,0.419410,0.880104,0.679519,NaN,0.396283,0.319619,0.262429,0.267048,Soft Threshold,0.419410,1.058358,1.312218,1.598181,1.570540
53,500,100,0.95,fro,S,0.518841,0.392081,0.973919,0.450736,NaN,0.475640,0.356634,0.274209,0.251642,Soft Threshold,0.392081,0.824324,1.099393,1.429863,1.558094
57,500,300,0.80,fro,S,0.609856,0.273069,0.883377,0.508406,NaN,0.335240,0.244207,0.171480,0.164404,Soft Threshold,0.273069,0.814547,1.118185,1.592425,1.660960
61,500,300,0.95,fro,S,0.314768,0.235500,0.973713,0.297102,NaN,0.408357,0.295583,0.202708,0.169351,Soft Threshold,0.235500,0.576703,0.796732,1.161774,1.390606
65,500,500,0.80,fro,S,0.475970,0.215561,0.883378,0.419988,NaN,0.320097,0.224181,0.142013,0.131291,Soft Threshold,0.215561,0.673424,0.961548,1.517894,1.641862
69,500,500,0.95,fro,S,0.233995,0.176260,0.973639,0.224958,NaN,0.240840,0.267434,0.172407,0.135612,Soft Threshold,0.176260,0.731856,0.659077,1.022348,1.299736


In [65]:
tmp = df[(df['ord'] == 'fro') & (df['type'] == 'S') & (df['N'] == 300)]
tmp

,N,T,rho,ord,type,Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink,Info Band eta=0.5,Info Band eta=0.8,Info Band eta=0.95,Info Band eta=1,min_other_key,min_other_val,ratio_eta=0.5,ratio_eta=0.8,ratio_eta=0.95,ratio_eta=1
25,300,100,0.80,fro,S,0.839914,0.359178,0.884476,0.581832,NaN,0.433204,0.357403,0.295806,0.295807,Soft Threshold,0.359178,0.829121,1.004968,1.214237,1.214231
29,300,100,0.95,fro,S,0.439947,0.285761,0.974010,0.349928,NaN,0.497606,0.394539,0.327522,0.309901,Soft Threshold,0.285761,0.574272,0.724291,0.872496,0.922104
33,300,300,0.80,fro,S,0.488942,0.230638,0.880438,0.410773,NaN,0.363705,0.264160,0.178208,0.172856,Soft Threshold,0.230638,0.634135,0.873100,1.294208,1.334281
37,300,300,0.95,fro,S,0.236401,0.176506,0.208291,0.218453,NaN,0.240447,0.270614,0.192253,0.160788,Soft Threshold,0.176506,0.734073,0.652240,0.918090,1.097756
41,300,500,0.80,fro,S,0.377207,0.377197,0.377207,0.336998,0.325845,0.345877,0.243921,0.147009,0.137967,Nonlinear Shrink,0.325845,0.942082,1.335859,2.216500,2.361762
45,300,500,0.95,fro,S,0.180067,0.180067,0.153661,0.168953,0.169377,0.180067,0.218724,0.158541,0.122933,Hard Threshold,0.153661,0.853355,0.702531,0.969214,1.249955


In [67]:
tmp = df[(df['ord'] == 'fro') & (df['type'] == 'S') & (df['N'] == 100)]
tmp

,N,T,rho,ord,type,Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink,Info Band eta=0.5,Info Band eta=0.8,Info Band eta=0.95,Info Band eta=1,min_other_key,min_other_val,ratio_eta=0.5,ratio_eta=0.8,ratio_eta=0.95,ratio_eta=1
1,100,100,0.80,fro,S,0.468781,0.423442,0.468781,0.419056,NaN,0.449804,0.365138,0.276803,0.277256,Linear Shrink,0.419056,0.931642,1.147666,1.513913,1.511442
5,100,100,0.95,fro,S,0.235507,0.239158,0.236136,0.243439,NaN,0.253561,0.323764,0.272303,0.260194,Sample,0.235507,0.928799,0.727404,0.864871,0.905122
9,100,300,0.80,fro,S,0.274187,0.274050,0.267349,0.256148,0.253258,0.354384,0.258011,0.172573,0.164797,Nonlinear Shrink,0.253258,0.714644,0.981580,1.467540,1.536793
13,100,300,0.95,fro,S,0.131696,0.131696,0.131696,0.119872,0.125182,0.131696,0.131696,0.148052,0.140719,Linear Shrink,0.119872,0.910217,0.910217,0.809659,0.851854
17,100,500,0.80,fro,S,0.207967,0.151988,0.167519,0.197419,0.195888,0.232465,0.217598,0.129592,0.121198,Soft Threshold,0.151988,0.653808,0.698480,1.172814,1.254042
21,100,500,0.95,fro,S,0.099378,0.090217,0.099378,0.092430,0.095949,0.099378,0.099378,0.110485,0.112550,Soft Threshold,0.090217,0.907816,0.907816,0.816552,0.801568
